# Power output for REZ for different energy scenarios

In [1]:
from dask.distributed import Client,LocalCluster
from dask_jobqueue import PBSCluster

In [46]:
# client.close()
# cluster.close()

In [52]:
# One node on Gadi has 48 cores - try and use up a full node before going to multiple nodes (jobs)

walltime = "00:10:00"
cores = 48
memory = str(4 * cores) + "GB"

cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory), processes=cores,
                     job_extra_directives=["-q normal",
                                           "-P dt6",
                                           "-l ncpus="+str(cores),
                                           "-l mem="+str(memory),
                                           "-l storage=gdata/w42+gdata/rt52+scratch/dt6"],
                     local_directory="$TMPDIR",
                     job_directives_skip=["select"])

/g/data/w42/dr6273/apps/conda/envs/pangeo/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 38931 instead
  warnings.warn(


In [53]:
cluster.scale(jobs=3)
client = Client(cluster)

In [54]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: /proxy/38931/status,
Dashboard: /proxy/38931/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.121.6:36563,Workers: 0
Dashboard: /proxy/38931/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [2]:
import xarray as xr
import pandas as pd

In [3]:
%cd /g/data/w42/dr6273/work/power_models
import functions as fn

/g/data/w42/dr6273/work/power_models


In [4]:
%load_ext autoreload
%autoreload 2

# Load REZ data

#### REZ mask

In [5]:
mask = xr.open_dataset('/g/data/w42/dr6273/work/projects/Aus_energy/data/rez_2024_mask_era5_grid.nc').REZ

#### REZ generation

In [6]:
_gen = pd.read_csv("/g/data/w42/dr6273/work/data/REZ/2024/REZ_potential.csv", index_col=0)

In [7]:
# Tidy column names
_gen.columns = [i.replace("90", "9-") for i in _gen.columns]
_gen.columns = [i.replace(" ", "_") for i in _gen.columns]

In [8]:
# Sum scenarios with existing capacity
gen = _gen.copy()
for col in _gen.columns[2:]:
    if col[:3] == "Exi":
        pass
    else:
        gen_type = col.split("_")[-2]
        gen[col] = _gen[col] + _gen["Existing_" + gen_type]

In [9]:
gen

,Solar_renewable_potential_(MW),Wind_renewable_potential_(MW),Existing_solar,Progressive_solar_2029-30,Progressive_solar_2039-40,Progressive_solar_2049-50,Step_change_solar_2029-30,Step_change_solar_2039-40,Step_change_solar_2049-50,Green_energy_solar_2029-30,...,Existing_wind,Progressive_wind_2029-30,Progressive_wind_2039-40,Progressive_wind_2049-50,Step_change_wind_2029-30,Step_change_wind_2039-40,Step_change_wind_2049-50,Green_energy_wind_2029-30,Green_energy_wind_2039-40,Green_energy_wind_2049-50
REZ,,,,,,,,,,,,,,,,,,,,,
N1,6385,0,166,216,216,266,166,216,266,216,...,0,0,0,0,0,0,0,0,0,-300
N2,2950,7400,855,905,1355,3855,855,3255,6255,3155,...,442,3492,4042,7842,3442,7842,7842,4992,9142,9592
N3,6850,3000,1497,3197,3447,7597,2947,5897,8347,4447,...,673,4373,4373,4873,6123,6123,8473,7373,7473,8173
N4,8000,5100,53,253,253,403,203,203,453,253,...,198,298,298,298,298,298,348,348,348,348
N5,2256,3900,1122,1822,1822,3372,2272,2272,3372,1922,...,0,450,450,450,600,600,1000,1000,1000,1000
N6,1028,1000,456,456,456,456,556,556,956,956,...,0,0,0,0,0,0,0,0,0,0
N7,0,0,0,0,0,0,0,0,0,0,...,270,270,270,270,270,270,270,270,270,270
N8,0,300,0,0,0,0,0,0,0,0,...,113,463,463,513,463,463,913,613,4063,4213
N9,516,1400,0,500,500,500,500,500,1000,650,...,0,550,550,1000,450,1000,1500,1100,2050,4400


### Artifically increase generation in some REZs

- Based on https://github.com/dougrichardson/seasonal_energy/blob/main/06_capacity_factors.ipynb
- Testing to see if adding capacity alters correlation of energy with climate modes

In [10]:
# Replace Step change 2049-50 wind in Q1 with green energy scenario
gen["artificial_Q1_wind_2049-50"] = gen["Step_change_wind_2049-50"].copy()
gen.loc["Q1", "artificial_Q1_wind_2049-50"] = gen.loc["Q1"]["Green_energy_wind_2049-50"].copy()

In [11]:
# Replace Step change 2049-50 wind in Q1 with green energy scenario
gen["artificial_Q2_wind_2049-50"] = gen["Step_change_wind_2049-50"].copy()
gen.loc["Q2", "artificial_Q2_wind_2049-50"] = gen.loc["Q2"]["Green_energy_wind_2049-50"].copy()

In [12]:
# Replace Step change 2049-50 wind in N10 with 5,200 MW (https://www.dcceew.gov.au/energy/renewable/offshore-wind/areas/hunter)
gen["artificial_N10_wind_2049-50"] = gen["Step_change_wind_2049-50"].copy()
gen.loc["N10", "artificial_N10_wind_2049-50"] = 5200

In [13]:
# Replace Step change 2049-50 wind in N11 with 2,900 MW (https://www.dcceew.gov.au/energy/renewable/offshore-wind/areas/illawarra)
gen["artificial_N11_wind_2049-50"] = gen["Step_change_wind_2049-50"].copy()
gen.loc["N11", "artificial_N11_wind_2049-50"] = 2900

In [14]:
# Replace Step change 2049-50 wind in N11 with 2,900 MW (https://www.dcceew.gov.au/energy/renewable/offshore-wind/areas/illawarra)
gen["artificial_V7_wind_2049-50"] = gen["Step_change_wind_2049-50"].copy()
gen.loc["V7", "artificial_V7_wind_2049-50"] = 2900

In [15]:
# Replace Step change 2049-50 wind in S6 with with renewable potential (2,400 MW)
gen["artificial_S6_wind_2049-50"] = gen["Step_change_wind_2049-50"].copy()
gen.loc["S6", "artificial_S6_wind_2049-50"] = 2400

In [16]:
# Replace Step change 2049-50 wind in S9 with green energy scenario
gen["artificial_S9_wind_2049-50"] = gen["Step_change_wind_2049-50"].copy()
gen.loc["S9", "artificial_S9_wind_2049-50"] = gen.loc["S9"]["Green_energy_wind_2049-50"].copy()

In [17]:
# Replace Step change 2049-50 wind in S10 with renewable potential (20,428 MW)
gen["artificial_S10_wind_2049-50"] = gen["Step_change_wind_2049-50"].copy()
gen.loc["S10", "artificial_S10_wind_2049-50"] = 20428

In [18]:
# Replace Step change 2049-50 wind in V4 with green energy scenario
gen["artificial_V4_wind_2049-50"] = gen["Step_change_wind_2049-50"].copy()
gen.loc["V4", "artificial_V4_wind_2049-50"] = gen.loc["V4"]["Green_energy_wind_2049-50"].copy()

In [19]:
gen.loc["V4"]

Solar_renewable_potential_(MW)       0
Wind_renewable_potential_(MW)     3442
Existing_solar                       0
Progressive_solar_2029-30            0
Progressive_solar_2039-40            0
Progressive_solar_2049-50            0
Step_change_solar_2029-30            0
Step_change_solar_2039-40            0
Step_change_solar_2049-50            0
Green_energy_solar_2029-30           0
Green_energy_solar_2039-40           0
Green_energy_solar_2049-50           0
Existing_wind                     3300
Progressive_wind_2029-30          3300
Progressive_wind_2039-40          3300
Progressive_wind_2049-50          4600
Step_change_wind_2029-30          4100
Step_change_wind_2039-40          4150
Step_change_wind_2049-50          6350
Green_energy_wind_2029-30         5350
Green_energy_wind_2039-40         5400
Green_energy_wind_2049-50         7800
artificial_Q1_wind_2049-50        6350
artificial_Q2_wind_2049-50        6350
artificial_N10_wind_2049-50       6350
artificial_N11_wind_2049-

### Pre-compute capacity factor averages:

In [20]:
def calculate_mean_cf(years, gen_type, method="van_der_Wiel", chunks=None):
    """
    Return mean capacity factor over all years
    
    gen_type: str, 'wind' or 'solar'
    method: str, method used to compute capacity factor, e.g. 'van_der_Wiel'
    chunks: dict, how to chunk. Default is None
    """
    da_list = []
    for year in years:
        cf = fn.load_hourly_cf(year, gen_type, method=method, chunks={"lat": 75, "lon": 100, "time": -1}).capacity_factor
        cf = cf.mean("time").expand_dims({"time": [year]})
        da_list.append(cf)
    cf = xr.concat(da_list, dim="time")
    return cf.mean("time")

In [21]:
fp = "/g/data/w42/dr6273/work/projects/Aus_energy/production_metrics/"

In [22]:
years = range(1940, 2024)

In [23]:
compute = False

In [24]:
if compute:
    for gen_type in ["wind", "solar"]:
        mean_cf = calculate_mean_cf(years, gen_type)
        mean_cf.to_dataset(name="capacity_factor").to_netcdf(
            fp + gen_type + "_mean_capacity_factor_van_der_Wiel_era5_hourly_" + str(years[0]) + "-" + str(years[-1]) + ".nc"
        )

# Compute generation

In [25]:
def get_even_capacity_mask(mask_da, generation):
    """
    Return DataArray with mask of capacity divided evenly across grid cells
    
    mask_da: array of REZ mask
    generation: pandas Series of capacity values for each region (index).
    """
    da_list = []
    for r in generation.index:
        capacity = generation.loc[r]
        n_cells = mask_da.sel(region=r).sum().values
        
        da = mask_da.sel(region=r).where(
            mask_da.sel(region=r) == 0,
            capacity / n_cells
        ).expand_dims({"REZ": [r]})
        
        da_list.append(da)
        
    return xr.concat(da_list, dim="REZ")

In [26]:
def get_cf_scaled_capacity_mask(cf_mean, mask_da, generation):
    """
    Return DataArray with mask of capacity divided according to weights from average capacity factor
    
    cf_mean: array of capacity factor weights
    mask_da: array of REZ mask
    generation: pandas Series of capacity values for each region (index).
    """
    da_list = []
    for r in generation.index:
        capacity = generation.loc[r]
        
        cf_region = cf_mean.where(
            mask_da.sel(region=r) == 1,
            drop=True
        )
        weights = cf_region / cf_region.sum()
        
        tolerance = 0.01
        if 1 - weights.sum().values > tolerance:
            print(weights.sum().values)
            raise ValueError("Weights don't sum to one.")
            
        da = (weights * capacity).expand_dims({"REZ": [r]})
        
        da_list.append(da)
        
    return xr.concat(da_list, dim="REZ")

In [27]:
def calc_generation(capacity_factor, generation_capacity):
    """
    Return DataArray with time series of power for each REZ
    
    capacity_factor: DataArray of capacity factors
    generation_capacity: DataArray of each grid cells generation capacity
    """
    da_list = []
    for r in generation_capacity.REZ.values:
        r_mask = generation_capacity.sel(REZ=r).where(
            generation_capacity.sel(REZ=r) > 0, drop=True
        )
        da = capacity_factor * r_mask
        da = da.sum(["lat", "lon"])
        da = da.expand_dims({"REZ": [r]})

        da_list.append(da)

    return xr.concat(da_list, dim="REZ")

In [28]:
def calc_all_years_generation(years, mask_da, generation, scale_mask="capacity_factor", method="van_der_Wiel", add_dir=""):
    """
    Compute power time series for each REZ and each year.
    
    years: range
    facilities: dict of facilities data in dataframe
    method: str, which method of capacity factors was used
    scale_mask: str, 'capacity_factor' to scale by mean capacity factors,
        'even' for an even distribution of generation amongst REZ cells,
        'none' to use mask_da directly.
    add_dir, add_dir: str, additional directory for load_hourly_cf, default ""
    """
    def _get_gen_type(gen):
        g_split = gen.name.split("_")
        if g_split[0] == "Solar":
            gt = "solar"
        elif g_split[0] == "Wind":
            gt = "wind"
        elif g_split[0] == "Existing":
            gt = g_split[1]
        else:
            gt = g_split[-2]
        return gt
    
    gen_type = _get_gen_type(generation)
    
    if scale_mask == "capacity_factor": # Use mean capacity factor weights to split the generation capacities by
        mean_cf = xr.open_dataset(
            "/g/data/w42/dr6273/work/projects/Aus_energy/production_metrics/" + \
            gen_type + "_mean_capacity_factor_van_der_Wiel_era5_hourly_" + str(years[0]) + "-" + str(years[-1]) + ".nc"
        )["capacity_factor"]
        
        gen_capacity = get_cf_scaled_capacity_mask(mean_cf, mask_da, generation)
        
    elif scale_mask == "even": # Divide the generation capacity by the size of the region and assign each cell that number
        gen_capacity = get_even_capacity_mask(mask_da, generation)
        
    else: # Just use mask_da as the generation capacities for each cell
        gen_capacity = mask_da.copy()
    
    da_list = []
    for year in years:
        cf = fn.load_hourly_cf(year, gen_type, method=method, add_dir=add_dir, chunks={"lat": -1, "lon": -1, "time": 2500}).capacity_factor
        p = calc_generation(cf, gen_capacity)
        da_list.append(p)
    REZ_power = xr.concat(da_list, dim="time")
    REZ_power = REZ_power.chunk({"REZ": -1, "time": -1})
    
    return REZ_power

In [34]:
gen.columns[[2, 6, 7, 8, 12, 16, 17, 18]]

Index(['Existing_solar', 'Step_change_solar_2029-30',
       'Step_change_solar_2039-40', 'Step_change_solar_2049-50',
       'Existing_wind', 'Step_change_wind_2029-30', 'Step_change_wind_2039-40',
       'Step_change_wind_2049-50'],
      dtype='object')

In [78]:
%%time
for scenario in gen.columns[[2, 6, 7, 8, 12, 16, 17, 18]]:
# for scenario in ["Existing_wind", "Existing_solar"]:
    print(scenario)
    gen_ds = calc_all_years_generation(years, mask, gen[scenario])
    gen_ds = gen_ds.to_dataset(name="power")
    
    # Tidy path to write
    if scenario.split("_")[-1] == "(MW)":
        scenario = scenario[:-5]
    scenario = scenario.lower()
    
    gen_ds.to_netcdf(
        "/scratch/dt6/dr6273/data/energy/" + "REZ_power_" + scenario + "_van_der_Wiel" + "_era5_hourly_" + str(years[0]) + "-" + str(years[-1]) + ".nc",
        mode="w"
    )

artificial_V4_wind_2049-50
CPU times: user 2min 16s, sys: 5.28 s, total: 2min 21s
Wall time: 2min 21s


### For reduced wind/solar

In [51]:
gen_ds

<xarray.Dataset> Size: 133MB
Dimensions:      (REZ: 43, time: 736344)
Coordinates:
  * REZ          (REZ) object 344B 'N1' 'N2' 'N3' 'N4' ... 'V5' 'V6' 'V7' 'V8'
  * time         (time) datetime64[ns] 6MB 1940-01-01 ... 2023-12-31T23:00:00
    region       (REZ) <U3 516B dask.array<chunksize=(43,), meta=np.ndarray>
    region_code  (REZ) int64 344B dask.array<chunksize=(43,), meta=np.ndarray>
    latitude     (REZ) float64 344B dask.array<chunksize=(43,), meta=np.ndarray>
    longitude    (REZ) float64 344B dask.array<chunksize=(43,), meta=np.ndarray>
Data variables:
    power        (REZ, time) float32 127MB dask.array<chunksize=(43, 736344), meta=np.ndarray>

In [55]:
%%time
for scenario in gen.columns[[17, 18]]:
# for scenario in ["Existing_wind", "Existing_solar"]:
    print(scenario)
    gen_ds = calc_all_years_generation(years, mask, gen[scenario], add_dir="reduced1pc/")
    gen_ds = gen_ds.to_dataset(name="power")
    
    # Tidy path to write
    if scenario.split("_")[-1] == "(MW)":
        scenario = scenario[:-5]
    scenario = scenario.lower()
    
    gen_ds.to_netcdf(
        "/scratch/dt6/dr6273/data/energy/" + "REZ_power_reduced_wind_solar_" + scenario + "_van_der_Wiel" + "_era5_hourly_" + str(years[0]) + "-" + str(years[-1]) + ".nc",
        mode="w"
    )

Step_change_wind_2039-40
Step_change_wind_2049-50
CPU times: user 4min 25s, sys: 14.7 s, total: 4min 40s
Wall time: 4min 49s


In [22]:
# %%time
# # for scenario in gen.columns[14:]:
# for scenario in ["Existing_wind", "Existing_solar"]:
#     print(scenario)
#     gen_ds = calc_all_years_generation(years, mask, gen[scenario], scale_mask="even")
#     gen_ds = gen_ds.to_dataset(name="power")
    
#     # Tidy path to write
#     if scenario.split("_")[-1] == "(MW)":
#         scenario = scenario[:-5]
#     scenario = scenario.lower()
    
#     gen_ds.to_netcdf(
#         fp + "REZ_power_even_weights_" + scenario + "_van_der_Wiel" + "_era5_hourly_" + str(years[0]) + "-" + str(years[-1]) + ".nc"
#     )

Existing_wind
Existing_solar
CPU times: user 3min 50s, sys: 12.2 s, total: 4min 2s
Wall time: 4min 29s


### Repeat but for existing facilities in sites outside REZs

In [56]:
solar_outside = xr.open_mfdataset(
    "/g/data/w42/dr6273/work/projects/Aus_energy/production_metrics/solar/power/solar_site_outside_REZ_mask_era5.nc",
).solar_site
# Change dimension name so it works with function
solar_outside = solar_outside.rename({"region": "REZ"}).compute()

In [57]:
wind_outside = xr.open_mfdataset(
    "/g/data/w42/dr6273/work/projects/Aus_energy/production_metrics/wind/power/wind_site_outside_REZ_mask_era5.nc",
).wind_site
wind_outside = wind_outside.rename({"region": "REZ"}).compute()

In [42]:
for scenario, m in zip(['Existing_solar', 'Existing_wind'], [solar_outside, wind_outside]):
    print(scenario)
    gen_ds = calc_all_years_generation(years, m, gen[scenario], scale_mask="none")
    gen_ds = gen_ds.to_dataset(name="power")
    
    # Tidy path to write
    if scenario.split("_")[-1] == "(MW)":
        scenario = scenario[:-5]
    scenario = scenario.lower()
    
    gen_ds = gen_ds.rename({"REZ": "region"}) # Change dimension name back
    
    gen_ds.to_netcdf(
        fp + "site_outside_REZ_power_" + scenario + "_van_der_Wiel" + "_era5_hourly_" + str(years[0]) + "-" + str(years[-1]) + ".nc",
        mode="w"
    )

Existing_solar
Existing_wind


### For reduced wind /solar

In [58]:
for scenario, m in zip(['Existing_solar', 'Existing_wind'], [solar_outside, wind_outside]):
    print(scenario)
    gen_ds = calc_all_years_generation(years, m, gen[scenario], scale_mask="none", add_dir="reduced1pc/")
    gen_ds = gen_ds.to_dataset(name="power")
    
    # Tidy path to write
    if scenario.split("_")[-1] == "(MW)":
        scenario = scenario[:-5]
    scenario = scenario.lower()
    
    gen_ds = gen_ds.rename({"REZ": "region"}) # Change dimension name back
    
    gen_ds.to_netcdf(
        fp + "site_outside_REZ_power_reduced_wind_solar_" + scenario + "_van_der_Wiel" + "_era5_hourly_" + str(years[0]) + "-" + str(years[-1]) + ".nc",
        mode="w"
    )

Existing_solar
Existing_wind


# Close cluster

In [59]:
client.close()
cluster.close()